# QFT Equivalence Check: Regular vs DQC

Verify equivalence between regular QFT and DQC QFT circuits using TDD.

In [ ]:
import numpy as np
import time
from qiskit import QuantumCircuit
from TDD2.TDD import Ini_TDD, Index
from TDD2.TDD_Q import cir_2_tn

In [ ]:
cir_regular = QuantumCircuit.from_qasm_file('qft_10.qasm')
display(cir_regular.draw('mpl'))
cir_dqc = QuantumCircuit.from_qasm_file('dynamic_qft_10.qasm')
# cir_dqc = QuantumCircuit.from_qasm_file('Benchmarks2/DQC_qft_5.qasm')
display(cir_dqc.draw('mpl'))

In [ ]:
# Load circuits
t_start = time.time()
cir_regular = QuantumCircuit.from_qasm_file('qft_10.qasm')
cir_dqc = QuantumCircuit.from_qasm_file('dynamic_qft_10.qasm')

# Convert to tensor networks
tn_regular, indices = cir_2_tn(cir_regular)
tn_dqc, indices_dqc = cir_2_tn(cir_dqc)

# Rename c indices to y in tn_dqc to match unitary output
for ts in tn_dqc.tensors:
    new_index_set = []
    for idx in ts.index_set:
        if idx.key.startswith('c'):
            # c{k}_0 -> y{k}
            parts = idx.key.split('_')
            k = parts[0][1:] # '1' or '0'
            new_key = 'y' + k
            new_index_set.append(Index(new_key, idx.idx))
        else:
            new_index_set.append(idx)
    ts.index_set = new_index_set

# Update indices list for DQC
indices_dqc_modified = []
for ts in tn_dqc.tensors:
    for idx in ts.index_set:
        if idx.key not in indices_dqc_modified:
            indices_dqc_modified.append(idx.key)

# Merge indices
for idx in indices_dqc_modified:
    if idx not in indices:
        indices.append(idx)

# Initialize TDD and contract
Ini_TDD(indices)
tdd_regular = tn_regular.cont(optimizer='tree_decomposition')[0]
tdd_dqc = tn_dqc.cont(optimizer='tree_decomposition')[0]

# Fix output indices of tdd_dqc (rename y{k} with idx > 0 to idx 0)
new_index_set = []
for idx in tdd_dqc.index_set:
    if idx.key.startswith('y') and idx.idx > 0:
        new_index_set.append(Index(idx.key, 0))
    else:
        new_index_set.append(idx)
tdd_dqc.index_set = new_index_set

# Check equivalence
equivalent = tdd_regular == tdd_dqc
elapsed = time.time() - t_start

print(f"Circuits Equivalent: {equivalent}")
print(f"Regular nodes: {tdd_regular.node_number()}")
print(f"DQC nodes: {tdd_dqc.node_number()}")
print(f"Time: {elapsed:.4f}s")

# tdd_regular.show()
# tdd_dqc.show()